In [1]:
import cv2
import numpy as np
from time import sleep
import matplotlib.pyplot as plt

In [2]:
PATH = "/Users/guyperets/Documents/IP_Course/YogaCoach/YogaExample3.mp4"
SMALL_CLIP_PATH = "/Users/guyperets/Downloads/SmallClip.mp4"
# Background Subtractor
backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)

In [3]:
cap = cv2.VideoCapture(PATH)
i = 0
# ? Define a ROI
while True:
    ret, frame = cap.read()
    if ret:
        i += 1
        if i < 900:
            continue
        if i == 900:
            print("Started IP...")
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(frame, (3, 3), 0)
        canny = cv2.Canny(blurred, 80, 240)
        backsub = backSub.apply(canny)
        # _, binary = cv2.threshold(backsub, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        dilated = cv2.dilate(backsub, (5, 5), iterations=10)
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask = np.zeros_like(frame)
        centroids = []
        for cnt in contours:
            # Connect the contours
            cv2.drawContours(mask, [cnt], -1, (255, 255, 255), cv2.FILLED)
            M = cv2.moments(cnt)
            if M['m00'] != 0:
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                centroids.append((cX, cY))
        close_centroids = []
        for c1 in centroids:
            for c2 in centroids:
                if c1 != c2:
                    if abs(c1[0] - c2[0]) > 20 and abs(c1[1] - c2[1]) < 20:
                        close_centroids.append((c1, c2))
        for close_pair in close_centroids:
            cv2.fillPoly(mask, np.array([close_pair]), (255, 255, 255))
            
        cv2.imshow("BackSub", mask)
        key = cv2.waitKey(0)
        if key & 0xFF == 27 or key & 0xFF == ord('q'): # ESC or 'q'
            break
    else:
        print("Video ended")
        break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)



Started IP...


-1

: 

In [ ]:
cap = cv2.VideoCapture(PATH)
backSub = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Video ended or failed to capture frame.")
        break

    # Apply background subtraction
    fgMask = backSub.apply(frame)

    # Threshold the foreground mask to get a binary image
    _, binary = cv2.threshold(fgMask, 200, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty mask with the same size as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Filter and draw contours on the mask
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:  # Filter small contours based on area
            cv2.drawContours(mask, [cnt], -1, (255, 255, 255), thickness=cv2.FILLED)

    # Convert the mask to grayscale and apply threshold to ensure binary mask
    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    _, mask_binary = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Display the original frame and the mask
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask_binary)

    # Wait for a key press and break the loop if 'q' or 'ESC' is pressed
    key = cv2.waitKey(30) & 0xFF
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


# Test Calibration

In [17]:
def track_object(roi):
    # Initialize the Kalman filter
    kf = cv2.KalmanFilter(4, 2, 0)
    kf.measurementMatrix = np.array([[1, 0], [0, 1]])
    kf.processNoiseCov = np.array([[0.1, 0], [0, 0.1]])
    kf.measurementNoiseCov = np.array([[0.01, 0], [0, 0.01]])

    # Get the current measurement of the object
    x, y = (roi.shape[1] // 2, roi.shape[0] // 2)
    measurement = np.array([x, y])

    # Update the Kalman filter with the current measurement
    kf.predict()
    kf.update(measurement)

    # Predict the next state of the object
    prediction = kf.predict()

    return prediction

In [1]:
def create_multi_tracker(joint_list):
    multi_tracker = cv2.legacy.MultiTracker_create()

    bboxes = []
    colors = [] 
    
    # OpenCV's selectROI function doesn't work for selecting multiple objects in Python
    # So we will call this function in a loop till we are done selecting all objects
    for i in range(len(joint_list)):
        # draw bounding boxes over objects
        # selectROI's default behaviour is to draw box starting from the center
        # when fromCenter is set to false, you can draw box starting from top left corner
        bbox = cv2.selectROI('MultiTracker', frame)
        bboxes.append(bbox)
        colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
        print("Press q to quit selecting boxes and start tracking")
        print("Press any other key to select next object")


    print(f'Selected bounding boxes: {bboxes}')

    for bbox in bboxes:
        multi_tracker.add(cv2.legacy.TrackerMOSSE_create(), frame, bbox)
        
    
    return multi_tracker

In [1]:
def normalize_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Normalize the frame to be in the range [0, 1]
    frame = frame.astype(np.float32)
    frame = (frame - frame.min()) / (frame.max() - frame.min())
    return frame

# Test Joint Tracking (Using OpenCV Built-In Trackers)
## Currently Working HERE

In [4]:
import cv2
import numpy as np
from time import sleep
import matplotlib.pyplot as plt
import sys
# from random import randint


cap = cv2.VideoCapture(0)
sleep(.5)
ret, frame = cap.read()
if not ret:
    print("Failed to capture frame.")
    sys.exit()
height, width, _ = frame.shape
print(f'Frame shape: {frame.shape}')
w = 100
h = 100
# joints = {"head": (width // 2 - w // 2, height // 4 - h // 2, 100, 100), # (x, y, w, h) - xy is the top left corner
#           "right_elbow": (width // 2 + w, height // 4 + h, 100, 100),
#           "left_elbow": (width // 2 - 2 * w, height // 4 + h, 100, 100),
#           "torso": (width // 2 - w // 2, height // 4 + h, 100, 100),
#           "right_knee": (width // 2 + int(1.5*w / 2) - 50, height // 4 + 3 * h, 100, 100),
#           "left_knee": (width // 2 - int(1.5*w / 2) - 50, height // 4 + 3 * h, 100, 100)}

joints = {"shoulder": (width // 2 - w // 2, height // 5, w, h), # (x, y, w, h) - xy is the top left corner
          "elbow": (width // 2 - 2 * w, height // 5, w, h),
          "wrist": (width // 2 - 4 * w, height // 5, w, h),
          "hip": (width // 2 - w // 2, height // 5 + 2 * h, w, h)}
        #   "knee": (width // 2 - w // 2, height // 5 + 3 * h, w, h),
        #   "ankle": (width // 2 - w // 2, height // 5 + 4 * h + 50, w, h)}
if not ret:
    print("Failed to capture frame.")

# Create a tracker for each joint
trackers = {joint: cv2.legacy.TrackerKCF_create() for joint in joints.keys()}

for joint, tracker in trackers.items():
    ret = tracker.init(frame, joints[joint])
    if not ret:
        print(f"Failed to initialize tracker for {joint}")

backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
i = 0
while True:
    i += 1
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        print("Failed to capture frame.")
        break
    if i < 500:
        # Apply background subtraction
        # fgMask = backSub.apply(frame)
        # Show each rectangle in the right position
        # cv2.rectangle(frame, joints["head"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["right_elbow"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["left_elbow"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["torso"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["right_knee"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["left_knee"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["shoulder"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["elbow"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["wrist"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["hip"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["knee"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["ankle"], (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break
        continue
    
    centroids = {}
    # success, boxes = multi_tracker.update(frame)
    # print(f'Updated tracker, success = {success}')
    # if success:
    #     for i, box in enumerate(boxes):
    #         print(f'Tracking box {list(joints.keys())[i]}: {box}')
    #         x, y, w, h = [int(v) for v in box]
    #         center_x = x + w // 2
    #         center_y = y + h // 2
    #         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    #         cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
    #         centroids[list(joints.keys())[i]] = (center_x, center_y)
    # else:
    #     cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    for joint, tracker in trackers.items():
        success, box = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in box]
            center_x = x + w // 2
            center_y = y + h // 2
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
            centroids[joint] = (center_x, center_y)
        else:
            cv2.putText(frame, f"Tracking failure detected for {joint}", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    
    # Connect the centroids with lines
    if len(list(centroids.values())) == len(joints.keys()):
        cv2.line(frame, centroids["shoulder"], centroids["elbow"], (0, 255, 0), 2)
        cv2.line(frame, centroids["elbow"], centroids["wrist"], (0, 255, 0), 2)
        # cv2.line(frame, centroids["hip"], centroids["knee"], (0, 255, 0), 2)
        # cv2.line(frame, centroids["knee"], centroids["ankle"], (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


Frame shape: (720, 1280, 3)


-1

: 

In [4]:
import cv2
import sys

# Set up tracker.
# Instead of MIL, you can also use

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'DaSiamRPN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[2]
i = 0
while i < len(tracker_types):
    for tracker_type in tracker_types:
        if tracker_type == 'BOOSTING':
            tracker = cv2.legacy.TrackerBoosting_create()
        # if tracker_type == 'MIL':
        #     tracker = cv2.TrackerMIL_create()
        # if tracker_type == 'KCF':
        #     tracker = cv2.TrackerKCF_create()
        elif tracker_type == 'TLD':
            tracker = cv2.legacy.TrackerTLD_create()
        elif tracker_type == 'MEDIANFLOW':
            tracker = cv2.legacy.TrackerMedianFlow_create()
        elif tracker_type == 'MOSSE':
            tracker = cv2.legacy.TrackerMOSSE_create()
        elif tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()
        else:
            continue

        # Read video
        video = cv2.VideoCapture(0)

        # Exit if video not opened.
        if not video.isOpened():
            print("Could not open video")
            sys.exit()

        # Read first frame.
        ok, frame = video.read()
        if not ok:
            print('Cannot read video file')
            sys.exit()
            
        # Define an initial bounding box
        bbox = (287, 23, 86, 320)

        # Uncomment the line below to select a different bounding box
        bbox = cv2.selectROI(frame, False)

        # Initialize tracker with first frame and bounding box
        ok = tracker.init(frame, bbox)

        while True:
            # Read a new frame
            ok, frame = video.read()
            if not ok:
                break
                
            # Start timer
            timer = cv2.getTickCount()

            # Update tracker
            ok, bbox = tracker.update(frame)

            # Calculate Frames per second (FPS)
            fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

            # Draw bounding box
            if ok:
                # Tracking success
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
            else :
                # Tracking failure
                cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

            # Display tracker type on frame
            cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
            
            # Display FPS on frame
            cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2)

            # Display result
            cv2.imshow("Tracking", frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == 27 or key == ord('q'):
                break
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        i += 1

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [4]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
height, width, _ = frame.shape
joints = {"head": (int(width * 0.25), int(height * 0.25), int(width * 0.3), int(height * 0.3)),
          "right_elbow": (int(width * 0.6), int(height * 0.6), int(width * 0.2), int(height * 0.3)),
          "left_elbow": (int(width * 0.2), int(height * 0.6), int(width * 0.2), int(height * 0.3))}
if not ret:
    print("Failed to capture frame.")
    exit()
# Initialize the tracker with the bounding box

tracker_head = cv2.legacy.TrackerMOSSE_create()
tracker_right_elbow = cv2.legacy.TrackerMOSSE_create()
tracker_left_elbow = cv2.legacy.TrackerMOSSE_create()

for joint, rect in joints.items():
    x, y, w, h = rect
    match joint:
        case "head":
            tracker_head.init(frame, (x, y, w, h))
        case "right_elbow":
            tracker_right_elbow.init(frame, (x, y, w, h))
        case "left_elbow":
            tracker_left_elbow.init(frame, (x, y, w, h))

backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
i = 0
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        print("Failed to capture frame.")
        break
    if i < 300:
        # Show each rectangle in the right position
        cv2.rectangle(frame, joints["head"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["right_elbow"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["left_elbow"], (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break
        i += 1
        continue
    centroids = {}
    for joint in joints.keys():
        match joint:
            case "head":
                rect = joints[joint]
                height, width, _ = frame.shape
                ret, bbox = tracker_head.update(frame)
                if ret:
                    # Tracking success
                    x, y, w, h = [int(v) for v in bbox]
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_x = x + w // 2
                    center_y = y + h // 2
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
                    centroids["head"] = (center_x, center_y)
                else:
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
            case "right_elbow":
                rect = joints[joint]
                height, width, _ = frame.shape
                ret, bbox = tracker_right_elbow.update(frame)
                if ret:
                    # Tracking success
                    x, y, w, h = [int(v) for v in bbox]
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_x = x + w // 2
                    center_y = y + h // 2
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
                    centroids["right_elbow"] = (center_x, center_y)
                else:
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
            case "left_elbow":
                rect = joints[joint]
                height, width, _ = frame.shape
                ret, bbox = tracker_left_elbow.update(frame)
                if ret:
                    # Tracking success
                    x, y, w, h = [int(v) for v in bbox]
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_x = x + w // 2
                    center_y = y + h // 2
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
                    centroids["left_elbow"] = (center_x, center_y)
                else:
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    # Connect the centroids with lines
    if len(centroids) == 3:
        cv2.line(frame, centroids["head"], centroids["right_elbow"], (0, 255, 0), 2)
        cv2.line(frame, centroids["head"], centroids["left_elbow"], (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1